In [1]:
# Imports 
import numpy as np
from english_words import english_words_set
import gym
from gym import spaces
from utils.exploration.base_exploration_model import BaseExplorationModel
from gym.utils.env_checker import check_env
from collections import deque
from utils.infrastructure.logger import Logger
from utils.infrastructure.schedule_utils import *

from collections import defaultdict
import re

In [32]:
class WordleSimple(gym.Env): 
    
    def __init__(self,
                 answer: str = None, 
                 valid_words: list = None, 
                 keep_answer_on_reset: bool = False, 
                 logdir: str = 'data'): 
        
        # Store attributes 
        assert(valid_words is not None), 'Must pass valid words'
        self.valid_words = valid_words
        self.n_valid_words = len(self.valid_words)
        self.answer = answer if answer is not None else np.random.choice(self.valid_words)
        self.keep_answer_on_reset = keep_answer_on_reset
        
        # Action + Observation Space
        self.action_space = gym.spaces.Discrete(self.n_valid_words)
        self.observation_space = gym.spaces.Box(low = 0, 
                                                high = 1, 
                                                shape = (self.n_valid_words,), 
                                                dtype = int)

        #  self.observation_space = gym.spaces.MultiDiscrete([2] * self.n_valid_words)
        
        # Init Stuff 
        self.state = np.ones(len(self.valid_words), dtype = int)
        self.guess_count = 0
        self.alphabet = list('abcdefghijklmnopqrstuvwxyz')
        self.possible_words = self.valid_words
        self.n_possible_words = len(self.possible_words)
        self.patterns = ['[abcdefghijklmnopqrstuvwxyz]']*5
        
        # Logging
        self.logging_freq = 500
        self.num_games = 0
        self.victory_buffer = deque(maxlen = self.logging_freq)
        self.win = False
        self.logger = Logger(logdir)   
        
        # Track yellow letters to enforce they are in the word 
        self.all_greens = set()
        self.all_yellows = set()
        self.all_grays = set()
        
    def _create_pattern(self, guess, answer: str = None): 
        
        answer = answer if answer is not None else self.answer
        
        # Init structures to check which letters are green and which are yellow
        greens = dict(zip(range(5), ['']*5))
        yellows = defaultdict(list)
        grays = []

        # Get which words belong where 
        for idx, (guess_letter, answer_letter) in enumerate(zip(guess, answer)): 
            if guess_letter == answer_letter: # green letter
                greens.update({idx: guess_letter})
                self.all_greens.add(guess_letter)
            elif guess_letter in answer: # yellow letter
                yellows[idx].append(guess_letter)
                self.all_yellows.add(guess_letter)
            else: 
                grays.append(guess_letter) # gray letter
                self.all_grays.add(guess_letter)

        # Remove grays from alphabet
        self.alphabet = sorted(set(self.alphabet) - set(grays))

        for i in range(5): 

            if greens[i] != '': 

                # If we have the green letter, we should just replace the whole pattern with this
                self.patterns[i] = '[' + greens[i] + ']'

            elif len(yellows[i])  > 0: 

                # If we get another yellow, remove it from the pattern 
                for letter in yellows[i]: 
                    self.patterns[i] = self.patterns[i].replace(letter, '')

            else: 
                self.patterns[i] = '[' + ''.join(self.alphabet) + ']'


        # Combine patterns into single new pattern 
        pattern = "".join(self.patterns)
        
        return pattern

    def _get_possible_words(self, pattern): 
        
        # Get possible words by matching regex
        new_possible_words = [word for word in self.possible_words if re.match(pattern, word)]

        # Enforce that all yellows are actually in the word -- regex not tracking it necessarily
        yellows = self.all_yellows - self.all_greens
        for letter in yellows: 
            new_possible_words = [word for word in new_possible_words if letter in word]
            
        return new_possible_words
    
    def _compute_reward(self, guess): 
        
        # Create pattern 
        pattern = self._create_pattern(guess)
        
        # Get possible words 
        new_possible_words = self._get_possible_words(pattern)
        print(new_possible_words)

        # Compute reward
        reward = (len(self.possible_words) - len(new_possible_words))/len(self.possible_words)
        
        # Check if won 
        win = bool(guess == self.answer)
        
        # Add win/loss penalty
        if win: 
            reward += 1
        else: 
            reward -= 1
        
        # Add possible word penalty 
        if guess not in self.possible_words: 
            reward -= 1
        

            
        return reward, win, new_possible_words
                
    def step(self, action): 
        
        # Grab decoded word 
        guess = self.valid_words[action]
        
        # Compute reward
        _, win, new_possible_words = self._compute_reward(guess)

        ### TRY GIVING THIS REWARD
        reward = 1 if guess in self.possible_words else -1
        
        # Update state
        self.state = np.array([1 if word in new_possible_words else 0 for word in self.valid_words], dtype=int)
        assert(self.state.shape == self.observation_space.shape), f'{self.state.shape}'
        self.possible_words = new_possible_words
        self.n_possible_words = len(self.possible_words)

        
        # Increment guess count 
        self.guess_count += 1
        
        # Check if done
        done = (win) or (self.guess_count == 6)
        
        # Info 
        info = {'guess_count': self.guess_count, 'won': win}
        
        self.win = win
                
        return self.state, reward, done, info
        
    def reset(self): 

        # Win Ratio logging
        self.victory_buffer.append(self.win)
        self.num_games += 1
        if not self.num_games % self.logging_freq:
            logs = {
                "win ratio": self._compute_win_ratio()
            }
            self.do_logging(logs, self.num_games)
    
       
        # Reset possible words = all valid words
        self.possible_words = self.valid_words
        self.patterns = ['[abcdefghijklmnopqrstuvwxyz]']*5
        self.all_greens = set()
        self.all_yellows = set()
        self.all_grays = set()
        
        # Reset alphabet, state and guess count
        self.alphabet = list('abcdefghijklmnopqrstuvwxyz')
        self.state = np.ones(len(self.valid_words), dtype = int)
        self.guess_count = 0

        # Reset answer 
        if not self.keep_answer_on_reset:
            self.answer = np.random.choice(self.valid_words)
        
        
        return self.state
 
    def _compute_win_ratio(self):
        """
        Computes the win ration of games currently in the victory buffer.
        :return: the win ratio
        """
        wins = sum(self.victory_buffer)
        return wins/len(self.victory_buffer)

    def do_logging(self, logs, num_games):
        """
        :param logs: dictionary containing values to be logged
        :param num_games: the number of games
        :return: logs values to tensorboard
        """
        print(f"Number of Games: {num_games}")
        print(f"State: \n {self.state}")
        print(f"Possible Words: \n {self.possible_words}")
        print(f"Answer: \n {self.answer}")
        for key, value in logs.items():
            print('{} : {}'.format(key, value))
            self.logger.log_scalar(value, key, num_games)
        print("\n")

In [33]:
words = [word.replace('\n', '').replace(',', '') for word in list(open('scripts/wordle_words.txt', 'r'))]

In [34]:
env = WordleSimple(valid_words = words, answer = 'roost')

########################
logging outputs to  data
########################


In [36]:
env._create_pattern('scoff', 'miser')

'[abcdefghijklmnopqrtuvwxyz][abdeghijklmnpqrstuvwxyz][abdeghijklmnpqrstuvwxyz][abdeghijklmnpqrstuvwxyz][abdeghijklmnpqrstuvwxyz]'

In [ ]:
for i in range(1000): 
    try: 
        obs = env.reset()
        print(f'Answer: {env.answer}\n')
        done = False
        i = 1
        while not done:
            # action, _states = agent.predict(obs)
            action = env.action_space.sample()
            print(f'Guess #{i}: {env.valid_words[action]}')
            obs, rewards, done, info = env.step(action)
            i += 1
    except: 
        print("FUCKED UP")

Answer: scald

Guess #1: brush
['aisle', 'ascot', 'aside', 'askew', 'assay', 'asset', 'caste', 'disco', 'easel', 'ensue', 'essay', 'fetus', 'ficus', 'fishy', 'focus', 'islet', 'issue', 'locus', 'mason', 'minus', 'nasal', 'nasty', 'nosey', 'onset', 'pasta', 'paste', 'pasty', 'pesky', 'pesto', 'posit', 'psalm', 'sadly', 'saint', 'salad', 'sally', 'salon', 'salty', 'salve', 'salvo', 'sandy', 'sappy', 'satin', 'savoy', 'savvy', 'scald', 'scale', 'scalp', 'scaly', 'scamp', 'scant', 'scare', 'scarf', 'scary', 'scene', 'scent', 'scion', 'scoff', 'scold', 'scone', 'scoop', 'scope', 'score', 'scorn', 'scout', 'scowl', 'sedan', 'seedy', 'segue', 'seize', 'semen', 'sepia', 'setup', 'seven', 'siege', 'sieve', 'sight', 'sigma', 'silky', 'silly', 'since', 'sinew', 'singe', 'sixty', 'skate', 'skiff', 'skill', 'skimp', 'skirt', 'slack', 'slain', 'slang', 'slant', 'slate', 'sleek', 'sleep', 'sleet', 'slept', 'slice', 'slick', 'slide', 'slime', 'slimy', 'sling', 'slink', 'sloop', 'slope', 'slyly', 'smac

Guess #2: silly
['alarm', 'album', 'alert', 'algae', 'alien', 'align', 'alike', 'alive', 'altar', 'alter', 'angel', 'annul', 'anvil', 'avail', 'axial', 'bagel', 'banal', 'basal', 'basil', 'betel', 'bevel', 'bezel', 'black', 'blame', 'blank', 'blare', 'blast', 'blaze', 'bleak', 'bleat', 'bleep', 'blimp', 'blink', 'blitz', 'bluer', 'bluff', 'blunt', 'blurb', 'blurt', 'cabal', 'camel', 'canal', 'cavil', 'clack', 'claim', 'clamp', 'clang', 'clank', 'clasp', 'clean', 'clear', 'cleat', 'cleft', 'clerk', 'click', 'cliff', 'climb', 'cling', 'clink', 'cluck', 'clump', 'clung', 'cruel', 'easel', 'elate', 'elect', 'elfin', 'elite', 'email', 'equal', 'excel', 'expel', 'fatal', 'fecal', 'feral', 'fetal', 'flack', 'flail', 'flair', 'flake', 'flame', 'flank', 'flare', 'flask', 'fleck', 'fleet', 'flick', 'flier', 'fling', 'flint', 'flirt', 'fluff', 'fluke', 'flume', 'flung', 'flunk', 'flute', 'frail', 'gavel', 'glare', 'glaze', 'gleam', 'glean', 'glint', 'grail', 'gruel', 'kneel', 'label', 'lager', 'l

Guess #2: wharf
['heard', 'hoard']
Guess #3: abhor
['heard']
Guess #4: tonga
['heard']
Guess #5: dried
['heard']
Guess #6: skill
['heard']
Answer: flick

Guess #1: infer
['cliff', 'edify', 'faith', 'ficus', 'field', 'fight', 'filly', 'filmy', 'filth', 'final', 'finch', 'first', 'fishy', 'fizzy', 'flail', 'flick', 'fluid', 'foist', 'folio', 'fungi', 'motif', 'shift', 'skiff', 'stiff', 'swift', 'whiff']
Guess #2: hunky
['flick', 'skiff']
Guess #3: claim
['flick']
Guess #4: eight
['flick']
Guess #5: umbra
['flick']
Guess #6: ramen
['flick']
Answer: pedal

Guess #1: maple
['leapt', 'pearl', 'pedal', 'penal', 'petal', 'plead', 'pleat']
Guess #2: dusky
['pedal', 'plead']
Guess #3: inane
['pedal', 'plead']
Guess #4: booze
['pedal', 'plead']
Guess #5: brick
['pedal', 'plead']
Guess #6: pesto
['pedal']
Answer: frock

Guess #1: parka
['break', 'brick', 'brink', 'brisk', 'brook', 'clerk', 'crack', 'crank', 'creak', 'creek', 'crick', 'croak', 'crock', 'crook', 'drank', 'drink', 'drunk', 'frank', '

Guess #2: crick
['defer', 'demur', 'deter', 'donor', 'dowry', 'egret', 'enter', 'entry', 'ester', 'ether', 'every', 'exert', 'femur', 'ferry', 'fever', 'fewer', 'fjord', 'forge', 'forgo', 'forte', 'forth', 'forty', 'forum', 'foyer', 'furor', 'furry', 'genre', 'goner', 'gorge', 'gourd', 'heron', 'homer', 'honor', 'horde', 'horny', 'horse', 'hover', 'humor', 'hurry', 'hydro', 'hyper', 'juror', 'merge', 'merry', 'meter', 'metro', 'moron', 'morph', 'motor', 'mourn', 'mover', 'mower', 'myrrh', 'nerdy', 'nerve', 'never', 'newer', 'north', 'nurse', 'odder', 'offer', 'other', 'otter', 'outer', 'overt', 'owner', 'poser', 'power', 'puree', 'purer', 'purge', 'purse', 'queer', 'query', 'reedy', 'refer', 'renew', 'rerun', 'reset', 'retro', 'retry', 'reuse', 'revue', 'rhyme', 'rodeo', 'roger', 'rogue', 'roomy', 'roost', 'rotor', 'rouge', 'rough', 'round', 'rouse', 'route', 'rover', 'rowdy', 'rower', 'ruddy', 'ruder', 'rumor', 'rupee', 'rusty', 'score', 'scorn', 'scour', 'scree', 'screw', 'scrum', 's

Guess #3: broth
['alien', 'angel', 'annex', 'apnea', 'clean', 'clued', 'decal', 'deign', 'devil', 'eking', 'elfin', 'ennui', 'equal', 'equip', 'excel', 'expel', 'fecal', 'feign', 'fella', 'field', 'fiend', 'fleck', 'gavel', 'given', 'glean', 'ideal', 'index', 'knead', 'kneed', 'kneel', 'laden', 'lapel', 'legal', 'level', 'liken', 'linen', 'navel', 'panel', 'pecan', 'pedal', 'penal', 'pixel', 'plead', 'plied', 'queen', 'quell', 'unfed', 'vegan', 'vixen']
Guess #4: drape
['clued', 'eking', 'elfin', 'ennui', 'equip', 'excel', 'feign', 'fella', 'field', 'fiend', 'fleck', 'given', 'kneed', 'kneel', 'level', 'liken', 'linen', 'queen', 'quell', 'unfed', 'vixen']
Guess #5: bluer
['excel', 'kneel', 'level', 'liken', 'linen']
Guess #6: again
['liken', 'linen']
Answer: tower

Guess #1: blood
['aorta', 'audio', 'cacao', 'cameo', 'cargo', 'cello', 'coach', 'coast', 'cobra', 'comet', 'comfy', 'comic', 'comma', 'conch', 'condo', 'conic', 'copse', 'corer', 'corny', 'couch', 'cough', 'count', 'coupe', 

...

Guess #2: whole
['abled', 'alert', 'alien', 'alter', 'baler', 'belch', 'betel', 'bevel', 'bezel', 'bleak', 'bleat', 'bleed', 'bleep', 'blend', 'bless', 'bluer', 'camel', 'clean', 'clear', 'cleat', 'clerk', 'clued', 'cruel', 'decal', 'delta', 'devil', 'easel', 'eclat', 'elder', 'elect', 'email', 'equal', 'excel', 'expel', 'ideal', 'idler', 'impel', 'inlet', 'islet', 'kneel', 'label', 'laden', 'lapel', 'later', 'leach', 'leant', 'leapt', 'learn', 'leash', 'least', 'leech', 'lemur', 'leper', 'level', 'lever', 'libel', 'liken', 'linen', 'liner', 'liver', 'lumen', 'medal', 'metal', 'navel', 'paler', 'panel', 'pearl', 'pedal', 'penal', 'peril', 'petal', 'pixel', 'plead', 'pleat', 'plied', 'plier', 'rebel', 'relax', 'relic', 'renal', 'repel', 'revel', 'ruler', 'sleek', 'sleep', 'sleet', 'slept', 'spiel', 'steal', 'steel', 'ulcer', 'valet']
Guess #3: pivot
['peril', 'plied', 'plier']
Guess #4: horse
['plied', 'plier']
Guess #5: safer
['plied']
Guess #6: lousy
['plied']
Answer: dully

Guess #1:

Guess #2: bribe
['admin', 'again', 'anvil', 'chain', 'cinch', 'conic', 'cynic', 'dingo', 'dingy', 'final', 'finch', 'inlay', 'ionic', 'kinky', 'lingo', 'login', 'mania', 'manic', 'minim', 'ninja', 'panic', 'pinch', 'pinky', 'plain', 'slain', 'snail', 'sonic', 'vinyl', 'winch', 'windy']
Guess #3: guild
['dingo', 'dingy']
Guess #4: snail
['dingo', 'dingy']
Guess #5: union
['dingo']
Guess #6: bench
['dingo']
Answer: turbo

Guess #1: tweet
['tabby', 'table', 'taboo', 'tacky', 'taffy', 'tally', 'talon', 'tango', 'tangy', 'tapir', 'tardy', 'taste', 'tasty', 'tatty', 'thank', 'thick', 'thigh', 'thing', 'think', 'third', 'thong', 'thorn', 'those', 'throb', 'throw', 'thrum', 'thumb', 'thump', 'thyme', 'tiara', 'tibia', 'tidal', 'tilde', 'timid', 'tipsy', 'titan', 'tithe', 'title', 'today', 'toddy', 'tonal', 'tonga', 'tonic', 'tooth', 'topaz', 'topic', 'torch', 'torso', 'torus', 'total', 'touch', 'tough', 'toxic', 'toxin', 'trace', 'track', 'trade', 'trail', 'train', 'tramp', 'trash', 'triad', '

Guess #2: canon
['blond', 'borne', 'bound', 'downy', 'drone', 'ebony', 'found', 'frond', 'gnome', 'knoll', 'mound', 'noble', 'nobly', 'nomad', 'noose', 'nosey', 'novel', 'ozone', 'round', 'snore', 'snowy', 'sound', 'wound', 'wrong', 'young']
Guess #3: unfit
['bound', 'found', 'mound', 'round', 'sound', 'wound', 'young']
Guess #4: alien
['bound', 'mound', 'round', 'sound', 'wound', 'young']
Guess #5: paddy
['bound', 'mound', 'round', 'sound', 'wound']
Guess #6: phony
['bound', 'mound', 'round', 'sound', 'wound']
Answer: flung

Guess #1: valet
['black', 'bland', 'blank', 'blend', 'bless', 'blimp', 'blind', 'blink', 'bliss', 'block', 'blond', 'blood', 'bloom', 'blown', 'bluff', 'blurb', 'blush', 'brawl', 'broil', 'build', 'burly', 'chalk', 'child', 'chili', 'chill', 'civil', 'clack', 'claim', 'clamp', 'clang', 'clank', 'clash', 'clasp', 'class', 'clerk', 'click', 'cliff', 'climb', 'cling', 'clink', 'clock', 'cloud', 'clown', 'cluck', 'clump', 'clung', 'could', 'coyly', 'crawl', 'curly', '

Guess #2: amply
['brawl', 'carol', 'coral', 'crawl', 'drawl', 'flora', 'labor', 'larva', 'molar', 'moral', 'mural', 'polar', 'solar', 'trawl', 'ultra', 'valor']
Guess #3: ulcer
['brawl', 'drawl', 'larva', 'trawl']
Guess #4: slump
['brawl', 'drawl', 'larva', 'trawl']
Guess #5: reply
['brawl', 'drawl', 'larva', 'trawl']
Guess #6: peach
['larva']
Answer: leafy

Guess #1: piney
['beady', 'beefy', 'belly', 'berry', 'decay', 'decoy', 'decry', 'delay', 'derby', 'early', 'ebony', 'elegy', 'essay', 'every', 'ferry', 'geeky', 'heady', 'heavy', 'hefty', 'jelly', 'jerky', 'jetty', 'leafy', 'leaky', 'leery', 'lefty', 'leggy', 'mealy', 'meaty', 'mercy', 'merry', 'query', 'ready', 'reedy', 'relay', 'retry', 'seedy', 'teary', 'teddy', 'testy', 'weary', 'weedy', 'zesty']
Guess #2: gooey
['beady', 'beefy', 'belly', 'berry', 'decay', 'decoy', 'decry', 'delay', 'derby', 'early', 'elegy', 'essay', 'every', 'ferry', 'heady', 'heavy', 'hefty', 'jelly', 'jerky', 'jetty', 'leafy', 'leaky', 'leery', 'lefty', 'm

['known', 'shown', 'snoop', 'spoon', 'swoon', 'thong']
Guess #6: lupus
['known', 'thong']
Answer: sieve

Guess #1: slant
['scoff', 'scoop', 'scope', 'score', 'scour', 'scree', 'screw', 'scrub', 'scrum', 'seedy', 'segue', 'seize', 'serif', 'serum', 'serve', 'sever', 'sewer', 'sheep', 'sheer', 'sheik', 'shied', 'shire', 'shirk', 'shock', 'shook', 'shore', 'shove', 'showy', 'shrew', 'shrub', 'shrug', 'shuck', 'shush', 'siege', 'sieve', 'sissy', 'skier', 'skiff', 'skimp', 'smirk', 'smock', 'smoke', 'smoky', 'sober', 'soggy', 'sorry', 'sower', 'speck', 'speed', 'sperm', 'spice', 'spicy', 'spied', 'spike', 'spiky', 'spire', 'spoke', 'spoof', 'spook', 'spore', 'spree', 'sprig', 'squib', 'super', 'surer', 'surge', 'sushi', 'sweep', 'swish', 'swoop', 'sword', 'swore', 'syrup']
Guess #2: ardor
['segue', 'seize', 'sheep', 'sheik', 'shuck', 'shush', 'siege', 'sieve', 'sissy', 'skiff', 'skimp', 'speck', 'spice', 'spicy', 'spike', 'spiky', 'squib', 'sushi', 'sweep', 'swish']
Guess #3: cacti
['seize'